In [ ]:
!pip install optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import optuna
import csv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving legal_text_classification.csv to legal_text_classification.csv


In [ ]:
# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]

In [ ]:
file_path = filename

In [ ]:
# Read the CSV file with error handling
try:
    data = pd.read_csv(file_path, quoting=csv.QUOTE_NONE, on_bad_lines='warn')
except pd.errors.ParserError as e:
    print(f"Error parsing the CSV file: {e}")
    data = pd.read_csv(file_path, quoting=csv.QUOTE_NONE, on_bad_lines='warn')

<ipython-input-8-55af1ed30ccd>:3: ParserWarning: Skipping line 3: expected 4 fields, saw 5
Skipping line 5: expected 4 fields, saw 5
Skipping line 6: expected 4 fields, saw 5
Skipping line 7: expected 4 fields, saw 12
Skipping line 8: expected 4 fields, saw 5
Skipping line 9: expected 4 fields, saw 16
Skipping line 10: expected 4 fields, saw 9
Skipping line 11: expected 4 fields, saw 11
Skipping line 12: expected 4 fields, saw 7
Skipping line 13: expected 4 fields, saw 23
Skipping line 14: expected 4 fields, saw 25
Skipping line 15: expected 4 fields, saw 14
Skipping line 16: expected 4 fields, saw 6
Skipping line 17: expected 4 fields, saw 38
Skipping line 18: expected 4 fields, saw 9
Skipping line 19: expected 4 fields, saw 31
Skipping line 20: expected 4 fields, saw 30
Skipping line 21: expected 4 fields, saw 10
Skipping line 22: expected 4 fields, saw 10
Skipping line 23: expected 4 fields, saw 7
Skipping line 24: expected 4 fields, saw 5
Skipping line 25: expected 4 fields, saw 17

In [ ]:
# Display the first few rows of the dataset
print(data.head())

# Check the distribution of labels
print(data['case_outcome'].value_counts())

# Encode the labels
label_encoder = LabelEncoder()
data['case_outcome'] = label_encoder.fit_transform(data['case_outcome'])

  case_id case_outcome                                         case_title  \
0   Case1        cited  Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Lt...   
1   Case3        cited  Colgate Palmolive Co v Cussons Pty Ltd (1993) ...   
2  Case29     followed  Elderslie Finance Corp Ltd v Australian Securi...   
3  Case41        cited  Australian Securities and Investments Commissi...   
4  Case42        cited  Australian Securities and Investments Commissi...   

                                           case_text  
0  Ordinarily that discretion will be exercised s...  
1  Ordinarily that discretion will be exercised s...  
2                                                NaN  
3  Numerous cases have held that s 911A of the Ac...  
4  Numerous cases have held that s 911A of the Ac...  
case_outcome
cited            530
referred to      256
applied           65
followed          62
considered        32
distinguished     14
discussed         14
affirmed          12
related            5
appro

In [ ]:
# Handle missing values in the 'case_text' column
data['case_text'].fillna('missing', inplace=True)

# Split the data into features and target
X = data['case_text']
y = data['case_outcome']

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

<ipython-input-10-d752b9806318>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['case_text'].fillna('missing', inplace=True)


In [ ]:
# Pad the sequences
max_len = 100
X_pad = pad_sequences(X_seq, maxlen=max_len, padding='post')

In [ ]:
# Function to optimize Random Forest hyperparameters
def rf_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    rf_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_index, test_index in kf.split(X_pad):
        X_train, X_test = X_pad[train_index], X_pad[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Handle class imbalance using RandomOverSampler
        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

        rf_model.fit(X_train_resampled, y_train_resampled)
        preds = rf_model.predict(X_test)
        scores.append(f1_score(y_test, preds, average='weighted'))

    return np.mean(scores)

In [ ]:
# Optimize Random Forest hyperparameters
rf_study = optuna.create_study(direction='maximize')
rf_study.optimize(rf_objective, n_trials=50)

print("Best Random Forest hyperparameters: ", rf_study.best_params)

[I 2025-04-01 14:50:55,807] A new study created in memory with name: no-name-cfba9aed-cd5c-47c1-abb9-b2cf69705a2f
[I 2025-04-01 14:51:10,046] Trial 0 finished with value: 0.5848180569656316 and parameters: {'n_estimators': 143, 'max_depth': 43}. Best is trial 0 with value: 0.5848180569656316.
[I 2025-04-01 14:51:21,078] Trial 1 finished with value: 0.5863571833552708 and parameters: {'n_estimators': 173, 'max_depth': 46}. Best is trial 1 with value: 0.5863571833552708.
[I 2025-04-01 14:51:32,922] Trial 2 finished with value: 0.5879537405250372 and parameters: {'n_estimators': 187, 'max_depth': 39}. Best is trial 2 with value: 0.5879537405250372.
[I 2025-04-01 14:51:43,549] Trial 3 finished with value: 0.5673965293710298 and parameters: {'n_estimators': 199, 'max_depth': 10}. Best is trial 2 with value: 0.5879537405250372.
[I 2025-04-01 14:51:52,986] Trial 4 finished with value: 0.5866732698409459 and parameters: {'n_estimators': 146, 'max_depth': 50}. Best is trial 2 with value: 0.5879

Best Random Forest hyperparameters:  {'n_estimators': 175, 'max_depth': 50}


In [ ]:
# Train the best Random Forest model
best_rf_model = RandomForestClassifier(**rf_study.best_params, random_state=42)

In [ ]:
# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(len(label_encoder.classes_), activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Compile the LSTM model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# K-Fold Cross-Validation for LSTM
kf = KFold(n_splits=5, shuffle=True, random_state=42)
lstm_scores = []
lstm_accuracy_list = []
lstm_precision_list = []
lstm_recall_list = []

for train_index, test_index in kf.split(X_pad):
    X_train, X_test = X_pad[train_index], X_pad[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Handle class imbalance using RandomOverSampler
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    lstm_model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    lstm_pred = lstm_model.predict(X_test)
    lstm_pred_classes = np.argmax(lstm_pred, axis=1)

    lstm_scores.append(f1_score(y_test, lstm_pred_classes, average='weighted'))
    lstm_accuracy_list.append(accuracy_score(y_test, lstm_pred_classes))
    lstm_precision_list.append(precision_score(y_test, lstm_pred_classes, average='weighted'))
    lstm_recall_list.append(recall_score(y_test, lstm_pred_classes, average='weighted'))

print(f"LSTM F1 Score: {np.mean(lstm_scores)}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
LSTM F1 Score: 0.8498558997367155


In [ ]:
# Define the stacking model
stacked_model = StackingClassifier(
    estimators=[('rf', best_rf_model), ('lstm', lstm_model)],
    final_estimator=RandomForestClassifier(n_estimators=100, random_state=42)
)

In [ ]:
# K-Fold Cross-Validation for Stacked Model
stacked_scores = []
stacked_accuracy_list = []
stacked_precision_list = []
stacked_recall_list = []

for train_index, test_index in kf.split(X_pad):
    X_train, X_test = X_pad[train_index], X_pad[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Handle class imbalance using RandomOverSampler
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

    # Train the Random Forest model
    best_rf_model.fit(X_train_resampled, y_train_resampled)

    # Train the LSTM model
    lstm_model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

    # Get predictions from both models
    rf_preds = best_rf_model.predict(X_test)
    lstm_preds = lstm_model.predict(X_test)
    lstm_preds_classes = np.argmax(lstm_preds, axis=1)

    # Stack predictions
    stacked_X = np.column_stack((rf_preds, lstm_preds_classes))

    # Train the stacking model
    stacked_model.final_estimator_.fit(stacked_X, y_test)



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [ ]:
    stacked_pred = stacked_model.final_estimator_.predict(stacked_X) # Use the trained final estimator for prediction
    stacked_scores.append(f1_score(y_test, stacked_pred, average='weighted'))
    stacked_accuracy_list.append(accuracy_score(y_test, stacked_pred))
    stacked_precision_list.append(precision_score(y_test, stacked_pred, average='weighted'))
    stacked_recall_list.append(recall_score(y_test, stacked_pred, average='weighted'))

print(f"Stacked Model F1 Score: {np.mean(stacked_scores)}")

Stacked Model F1 Score: 0.9098403584127838


In [ ]:
# Create a DataFrame to compare metrics
metrics_df = pd.DataFrame({
    'Model': ['Random Forest', 'LSTM', 'Stacked Ensemble'],
    'Accuracy': [np.mean([f1_score(y[test_index], best_rf_model.predict(X_pad[test_index]), average='weighted') for train_index, test_index in kf.split(X_pad)]),
                        np.mean(lstm_accuracy_list),
                        np.mean(stacked_accuracy_list)],
    'Precision': [np.mean([precision_score(y[test_index], best_rf_model.predict(X_pad[test_index]), average='weighted') for train_index, test_index in kf.split(X_pad)]),
                          np.mean(lstm_precision_list),
                          np.mean(stacked_precision_list)],
    'Recall': [np.mean([recall_score(y[test_index], best_rf_model.predict(X_pad[test_index]), average='weighted') for train_index, test_index in kf.split(X_pad)]),
                       np.mean(lstm_recall_list),
                       np.mean(stacked_recall_list)],
    'F1 Score': [np.mean([f1_score(y[test_index], best_rf_model.predict(X_pad[test_index]), average='weighted') for train_index, test_index in kf.split(X_pad)]),
                        np.mean(lstm_scores),
                        np.mean(stacked_scores)]
})

# Display the metrics
print(metrics_df)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              Model  Accuracy  Precision    Recall  F1 Score
0     Random Forest  0.803944   0.891481  0.756926  0.803944
1              LSTM  0.799406   0.931522  0.799406  0.849856
2  Stacked Ensemble  0.914141   0.927656  0.914141  0.909840
